In [2]:
from sklearn.neighbors import KNeighborsClassifier
import os
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import GloVeFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import FastTextFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import USEFeatureGeneration
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import csv
import numpy as np
import pandas as pd
import collections
from imblearn.over_sampling import SMOTE

In [3]:
class KNN:
    def __init__(self, x_train, y_train, x_test, y_test, n):
        self.knn = KNeighborsClassifier(n_neighbors=n)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def train(self):
        self.knn.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.knn.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [4]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']
column_headings = ["Morbidity Class", "KNN1_Macro F1", "KNN1_Micro F1", "KNN5_Macro F1", "KNN5_Micro F1"]

with open("./results/word-embeddings-features/performance_KNN_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

In [5]:
all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Asthma
For n=1, Macro F1 score: 0.9580730380730381 and Micro F1 Score 0.9596153846153845
For n=5, Macro F1 score: 0.8820216384922268 and Micro F1 Score 0.8884615384615385
CAD
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
CAD
For n=1, Macro F1 score: 0.5139862914862915 and Micro F1 Score 0.5428571428571428
For n=5, Macro F1 score: 0.5124603174603174 and Micro F1 Score 0.5285714285714286
CHF
(12, 494, 300) (12,) Counter({1.0: 12})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Depression
For n=1, Macro F1 score: 0.9072420634920636 and Micro F1 Score 0.9128787878787878
For n=5, Macro F1 score: 0.8819173881673882 and Micro F1 Score 0.8871212121212121
Diabetes
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Diabetes
For n=1, Macro F1 score: 0.5951587301587302 and Micro F1 Score 0.6285714285714284
For n=5, Macro F1 score: 0.486

In [6]:
with open("./results/word-embeddings-features/performance_KNN_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = GloVeFeatureGeneration(train_preprocessed_df, morbidity).glove_matrix_gen()
    X = np.average(X, axis=1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
Asthma
For n=1, Macro F1 score: 0.9494898647104529 and Micro F1 Score 0.9525641025641025
For n=5, Macro F1 score: 0.891895751307516 and Micro F1 Score 0.8980769230769232
CAD
CAD
For n=1, Macro F1 score: 0.7626190476190475 and Micro F1 Score 0.7714285714285714
For n=5, Macro F1 score: 0.8048809523809524 and Micro F1 Score 0.8142857142857143
CHF
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
Depression
For n=1, Macro F1 score: 0.819404761904762 and Micro F1 Score 0.8348484848484847
For n=5, Macro F1 score: 0.7563368575868575 and Micro F1 Score 0.7840909090909092
Diabetes
Diabetes
For n=1, Macro F1 score: 0.583888888888889 and Micro F1 Score 0.6142857142857143
For n=5, Macro F1 score: 0.47603174603174603 and Micro F1 Score 0.5142857142857142
Gallstones
Gallstones
For n=1, Macro F1 score: 0.8272687035187035 and Micro F1 Score 0.8386363636363636
For n=5, Macro F1 score: 0.6591027028527028 and Micro F1 Score 0.696212121212121

In [7]:
with open("./results/word-embeddings-features/performance_KNN_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FastTextFeatureGeneration(train_preprocessed_df, morbidity).fasttext_matrix_gen()
    X = np.average(X, axis=1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Asthma
For n=1, Macro F1 score: 0.7319446892976305 and Micro F1 Score 0.7448717948717949
For n=5, Macro F1 score: 0.7504005390034802 and Micro F1 Score 0.7615384615384617
CAD
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
CAD
For n=1, Macro F1 score: 0.7030952380952381 and Micro F1 Score 0.7285714285714284
For n=5, Macro F1 score: 0.6292063492063491 and Micro F1 Score 0.6571428571428571
CHF
(12, 494, 300) (12,) Counter({1.0: 12})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Depression
For n=1, Macro F1 score: 0.7701548451548452 and Micro F1 Score 0.7765151515151516
For n=5, Macro F1 score: 0.7591969141969142 and Micro F1 Score 0.7696969696969698
Diabetes
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Diabetes
For n=1, Macro F1 score: 0.5099206349206349 and Micro F1 Score 0.5428571428571428
For n=5, Macro F1 score: 0.550

In [8]:
with open("./results/word-embeddings-features/performance_KNN_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = USEFeatureGeneration(train_preprocessed_df, morbidity).use_matrix_gen()
    X = np.average(X, axis=1)
    X = X.reshape(-1, 1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma


2023-05-04 14:31:56.216960: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(66, 512) (66,) Counter({0.0: 63, 1.0: 3})
Asthma
For n=1, Macro F1 score: 0.8542939087056732 and Micro F1 Score 0.8583333333333334
For n=5, Macro F1 score: 0.9036600899100898 and Micro F1 Score 0.9051282051282051
CAD
(62, 512) (62,) Counter({0.0: 35, 1.0: 27})
CAD
For n=1, Macro F1 score: 0.6570238095238095 and Micro F1 Score 0.6857142857142857
For n=5, Macro F1 score: 0.6283730158730158 and Micro F1 Score 0.6428571428571428
CHF
(12, 512) (12,) Counter({1.0: 12})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(66, 512) (66,) Counter({0.0: 58, 1.0: 8})
Depression
For n=1, Macro F1 score: 0.7849204961704962 and Micro F1 Score 0.7924242424242425
For n=5, Macro F1 score: 0.7647554324760206 and Micro F1 Score 0.7742424242424243
Diabetes
(63, 512) (63,) Counter({1.0: 35, 0.0: 28})
Diabetes
For n=1, Macro F1 score: 0.5922150072150072 and Micro F1 Score 0.6285714285714286
For n=5, Macro F1 score: 0.6135064935064934 and Micro F1 Score